## Linking without deduplication

A simple record linkage model using the `link_only` [link type](https://moj-analytical-services.github.io/splink/settings_dict_guide.html#link_type).

With `link_only`, only between-dataset record comparisons are generated. No within-dataset record comparisons are created, meaning that the model does not attempt to find within-dataset duplicates.


<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_dev/docs/demos/examples/duckdb/link_only.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_dev

In [2]:
from splink import splink_datasets

df = splink_datasets.fake_1000

# Split a simple dataset into two, separate datasets which can be linked together.
df_l = df.sample(frac=0.5)
df_r = df.drop(df_l.index)

df_l.head(2)

,unique_id,first_name,surname,dob,city,email,cluster
930,930,Luke,Robinnso,1981-10-18,Coventry,lrobinson@wolf.org,233
385,385,Lottie,Davis,1972-06-12,NaN,lottie.d7@morgan-pierce.com,100


In [3]:
import splink.comparison_library as cl
import splink.comparison_template_library as ctl
from splink import DuckDBAPI, Linker, SettingsCreator, block_on

settings = SettingsCreator(
    link_type="link_only",
    blocking_rules_to_generate_predictions=[
        block_on("first_name"),
        block_on("surname"),
    ],
    comparisons=[
        ctl.NameComparison(
            "first_name",
        ),
        ctl.NameComparison("surname"),
        ctl.DateComparison(
            "dob",
            input_is_string=True,
            invalid_dates_as_null=True,
            datetime_metrics=["month", "year", "year"],
            datetime_thresholds=[1, 1, 10],
        ),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        ctl.EmailComparison("email", include_username_fuzzy_level=False),
    ],
)

linker = Linker(
    [df_l, df_r],
    settings,
    database_api=DuckDBAPI(),
    input_table_aliases=["df_left", "df_right"],
)

In [4]:
from splink.exploratory import completeness_chart

completeness_chart(
    [df_l, df_r],
    cols=["first_name", "surname", "dob", "city", "email"],
    db_api=DuckDBAPI(),
    table_names_for_chart=["df_left", "df_right"],
)

alt.LayerChart(...)

In [5]:

deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    block_on("email"),
]


linker.training.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)

Probability two random records match is estimated to be  0.00346.
This means that amongst all possible pairwise record comparisons, one in 288.78 are expected to match.  With 250,000 total possible comparisons, we expect a total of around 865.71 matching pairs


In [6]:
linker.training.estimate_u_using_random_sampling(max_pairs=1e6, seed=1)

You are using the default value for `max_pairs`, which may be too small and thus lead to inaccurate estimates for your model's u-parameters. Consider increasing to 1e8 or 1e9, which will result in more accurate estimates, but with a longer run time.


----- Estimating u probabilities using random sampling -----



Estimated u probabilities using random sampling



Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [7]:
session_dob = linker.training.estimate_parameters_using_expectation_maximisation(block_on("dob"))
session_email = linker.training.estimate_parameters_using_expectation_maximisation(
    block_on("email")
)
session_first_name = linker.training.estimate_parameters_using_expectation_maximisation(
    block_on("first_name")
)


----- Starting EM training session -----



Estimating the m probabilities of the model by blocking on:
l."dob" = r."dob"

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob


Iteration 1: Largest change in params was -0.387 in the m_probability of surname, level `Exact match on surname`


Iteration 2: Largest change in params was 0.113 in probability_two_random_records_match


Iteration 3: Largest change in params was 0.0347 in probability_two_random_records_match


...

Iteration 3: Largest change in params was -0.00119 in the m_probability of surname, level `All other comparisons`


Iteration 4: Largest change in params was -0.000228 in the m_probability of surname, level `All other comparisons`


Iteration 5: Largest change in params was -4.89e-05 in the m_probability of surname, level `All other comparisons`



EM converged after 5 iterations



Your model is fully trained. All comparisons have at least one estimate for their m and u values


In [8]:
results = linker.inference.predict(threshold_match_probability=0.9)

In [9]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,surname_l,...,dob_l,dob_r,gamma_dob,city_l,city_r,gamma_city,email_l,email_r,gamma_email,match_key
0,3.20162,0.90196,df_left,df_right,445,444,Jacob,Jacob,3,Campbell,...,1988-06-05,1997-05-04,1,Lonon,London,0,j.c65@ortiz.com,None,-1,0
1,3.20162,0.90196,df_left,df_right,774,778,Armstrong,Armstrong,3,Eva,...,2027-04-21,2017-04-23,1,Peterborouhg,Peterbotrough,0,e.armstrong16odonnell.info,None,-1,0
2,3.20162,0.90196,df_left,df_right,239,242,Freya,Freya,3,Shah,...,1972-01-17,1970-12-17,1,London,Lonnod,0,f.s@flynn.com,None,-1,0
3,3.20162,0.90196,df_left,df_right,833,834,Mason,Mason,3,Smith,...,1983-03-16,1993-03-13,1,Kingston-uponH-ull,Kingston-upon-Hull,0,masons52@reed.com,None,-1,0
4,3.20162,0.90196,df_left,df_right,439,444,Jacob,Jacob,3,Campbell,...,1987-06-06,1997-05-04,1,Lonnod,London,0,None,None,-1,0
